In [1]:
#Which week of the NFl season are we in?
week = 10

In [2]:
import pandas as pd
import numpy as np
from fuzzywuzzy import process
from sqlalchemy import create_engine
from sklearn.preprocessing import StandardScaler

#connect to database
engine = create_engine('sqlite:///nfl_projections_db.sqlite')

# Point values for: PassYds, PassTD, Int, RushYd, RushTD, PassYd, PassTD
DK_SCORING_VECTOR = np.array([0.05,4.,-1.,0.1,6.,1.,0.1,6.])


#translate nicknames from swish to team codes from fantasy data
TEAM_NAME_DICT = {'49ers':'SF', 'Bears':'CHI', 'Bengals':'CIN', 'Bills':'BUF', 'Broncos':'DEN',
                  'Browns':'CLE','Buccaneers':'TB', 'Cardinals':'ARI', 'Chargers':'SD',
                  'Chiefs':'KC', 'Colts':'IND', 'Cowboys':'DAL', 'Dolphins':'MIA', 'Eagles':'PHI',
                  'Falcons':'ATL', 'Giants':'NYG', 'Jaguars':'JAX', 'Jets':'NYJ', 'Lions':'DET',
                  'Packers':'GB', 'Panthers':'CAR', 'Patriots':'NE', 'Raiders':'OAK', 'Rams':'LA',
                  'Ravens':'BAL','Redskins':'WAS', 'Saints':'NO', 'Seahawks':'SEA', 'Steelers':'PIT',
                  'Texans':'HOU', 'Titans':'TEN', 'Vikings':'MIN', 
                  'ARI': 'ARI', 'ATL': 'ATL', 'BAL': 'BAL', 'BUF': 'BUF', 'CAR': 'CAR',
                  'CHI': 'CHI', 'CIN': 'CIN', 'CLE': 'CLE', 'DAL': 'DAL', 'DEN': 'DEN',
                  'DET': 'DET', 'GB': 'GB', 'HOU': 'HOU', 'IND': 'IND', 'JAC': 'JAX', 'JAX':'JAX',
                  'KC': 'KC', 'LA': 'LA', 'MIA': 'MIA', 'MIN': 'MIN', 'NE': 'NE',
                  'NO': 'NO', 'NYG': 'NYG', 'NYJ': 'NYJ', 'OAK': 'OAK', 'PHI': 'PHI',
                  'PIT': 'PIT', 'SD': 'SD', 'SEA': 'SEA', 'SF': 'SF', 'TB': 'TB',
                  'TEN': 'TEN', 'WAS': 'WAS', 'GBP': 'GB', 'KCC': 'KC', 'NEP': 'NE', 'NOS': 'NO', 
                  'RAM': 'LA', 'SDC': 'SD', 'SFO': 'SF', 'TBB': 'TB'}

convert_team_name = lambda x: TEAM_NAME_DICT[x]
match_name = lambda x: str(process.extractOne(str(x), player_names)[0])
        
def flip_first_last(s):
    comma = s.find(",")
    return s[comma + 2:] + " " + s[:comma]


def standardize_names(df):
    # Check to make sure all names are correct and standard. Takes in a df
    # Will return True if all names are correct. False otherwise.
    for name in list(df['PlayerName'].unique()):
        if name in player_names:
            names_correct = True
        else:
            names_correct = False
            break
    if not names_correct:
        df['PlayerName'] = df['PlayerName'].apply(match_name)
    df['TeamName'] = df['TeamName'].apply(convert_team_name)
    df.index = df[['PlayerName','TeamName','Position']]
    del [df['PlayerName'],df['TeamName'],df['Position']]
    return df   


# Get a list of all the names for the current weeK
names = pd.read_sql_query("SELECT Name from fantasy_data_off_week_{}".format(week), engine).values
player_names = [i for [i] in names]

C:\Users\JoGa\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:35: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
######################################################################
# QBs
######################################################################

In [4]:

cols = 'name as PlayerName, primary_pos_abbr as Position, nickname as TeamName, home || " " || opp_abbr as Opponent,\
            dk_fpts as DKPts, dk_salary as DKSalary, fd_fpts as FDPts, fd_salary as FDSalary'

table_name = 'swish_week_{}'.format(week)

swish_qb = pd.read_sql_query("SELECT {s}\
    FROM {t} \
    WHERE primary_pos_abbr == 'QB' AND dk_fpts > 5 \
    ORDER BY PlayerName DESC".format(s=cols, t=table_name), engine)
        
swish_qb = standardize_names(swish_qb)

swish_qb.head()

,Opponent,DKPts,DKSalary,FDPts,FDSalary
"(Trevor Siemian, DEN, QB)",@ NO,17.6739,5600.0,16.5596,7200.0
"(Tom Brady, NE, QB)",vs SEA,18.3750,7400.0,17.1655,8300.0
"(Sam Bradford, MIN, QB)",@ WAS,13.4812,5000.0,12.7186,7000.0
"(Ryan Tannehill, MIA, QB)",@ SD,16.2807,5300.0,15.4437,7200.0
"(Ryan Fitzpatrick, NYJ, QB)",vs LA,17.6426,5200.0,16.7804,7200.0


In [5]:
cols = 'Name as PlayerName, Position, Team as TeamName, \
            PassingAttempts as PaAtt, PassingCompletions as Cmp, PassingYards as PaYds, PassingInterceptions as Int, \
            PassingTouchdowns as PaTD, RushingAttempts as RuAtt, RushingYards as RuYds, RushingTouchdowns as RuTD, \
            FumblesLost as FL, FantasyPointsDraftKings as DKPts, FantasyPointsFanDuel as FDPts'

table_name = 'fantasy_data_off_week_{}'.format(week)

fd_qb = pd.read_sql_query("SELECT {s}\
    FROM {t} \
    WHERE Position == 'QB' AND FantasyPointsDraftKings > 5 \
    ORDER BY PlayerName DESC".format(s=cols, t=table_name), engine)

fd_qb = standardize_names(fd_qb)

fd_qb.head()

,PaAtt,Cmp,PaYds,Int,PaTD,RuAtt,RuYds,RuTD,FL,DKPts,FDPts
"(Trevor Siemian, DEN, QB)",40.0,25.5,287.9,1.0,1.7,3.2,13.3,0.1,0.3,18.9,18.6
"(Tom Brady, NE, QB)",39.8,26.5,302.6,0.8,2.4,3.6,12.4,0.1,0.2,22.5,22.2
"(Sam Bradford, MIN, QB)",35.3,23.3,280.6,0.9,1.6,2.4,6.6,0.0,0.3,17.0,16.6
"(Ryan Tannehill, MIA, QB)",35.5,22.5,227.7,1.1,1.5,4.1,18.7,0.1,0.2,16.2,15.9
"(Ryan Fitzpatrick, NYJ, QB)",39.0,22.9,249.3,1.1,1.4,3.4,13.0,0.1,0.2,16.0,15.8


In [6]:
# FPts are PPR points in the db. Roughly equal to DK scoring
cols = 'Player as PlayerName, Team as TeamName, ATT as PaAtt, CMP as Cmp, YDS as PaYds, INTS as Int, TDS as PaTD, ATTz as RuAtt, \
        YDSz as RuYds, TDSz as RuTD, FL, FPTS as DKPts'

table_name = 'fantasy_pros_qb_week_{}'.format(week)

fp_qb = pd.read_sql_query("SELECT {s}\
    FROM {t} \
    WHERE FPTS > 5 \
    ORDER BY PlayerName DESC".format(s=cols, t=table_name), engine)

fp_qb['Position'] = 'QB'
fp_qb = standardize_names(fp_qb)

fp_qb.head()

,PaAtt,Cmp,PaYds,Int,PaTD,RuAtt,RuYds,RuTD,FL,DKPts
"(Trevor Siemian, DEN, QB)",36.4,22.4,259.9,1.0,1.7,1.8,5.6,0.1,0.3,15.4
"(Tom Brady, NE, QB)",37.2,24.1,277.3,0.6,2.0,2.3,4.9,0.1,0.2,18.4
"(Sam Bradford, MIN, QB)",34.7,22.1,240.6,0.6,1.3,1.5,3.3,0.0,0.2,13.4
"(Ryan Tannehill, MIA, QB)",33.1,20.3,226.6,1.0,1.3,2.8,11.8,0.1,0.2,13.7
"(Ryan Fitzpatrick, NYJ, QB)",34.3,20.5,226.9,1.1,1.2,3.1,12.4,0.1,0.3,12.8


In [7]:
cols = 'Name as PlayerName, Team as TeamName, Pos as Position, \
        Comp as Cmp, PassYards as PaYds, Int, PassTD as PaTD, \
        RushYards as RuYds, RushTD as RuTD, Att as RuAtt '

table_name = 'fantasy_sharks_week_{}'.format(week)

fs_qb = pd.read_sql_query("SELECT {s}\
    FROM {t} \
    WHERE FantasyPoints > 4 AND Pos == 'QB' \
    ORDER BY PlayerName DESC".format(s=cols, t=table_name), engine)

fs_qb['PlayerName'] = fs_qb['PlayerName'].apply(flip_first_last)
fs_qb = standardize_names(fs_qb)

fs_qb.head()

,Cmp,PaYds,Int,PaTD,RuYds,RuTD,RuAtt
"(Jameis Winston, TB, QB)",18.00,250,1.10,1.50,9,0.18,2.56
"(Russell Wilson, SEA, QB)",19.72,244,0.56,1.42,21,0.12,5.05
"(Carson Wentz, PHI, QB)",24.79,266,0.72,1.23,4,0.01,2.44
"(Ryan Tannehill, MIA, QB)",20.27,230,0.79,1.15,10,0.09,2.70
"(Alex Smith, KC, QB)",20.00,220,0.34,0.91,11,0.07,2.69


In [8]:
# These columns are common to all three dfs: ['Cmp', 'PaYds', 'Int', 'PaTD', 'RuAtt', 'RuYds', 'RuTD']
common_to_3 = list(fs_qb.columns)

# add the columns together which they have in common, drop any NaNs which should be shiite players
# also weight each accordingly, fp_qb gets 5 times the weight cuz its an aggregate of 5 sources
df3_qb = (fs_qb[common_to_3]/7 + fd_qb[common_to_3]/7 + fp_qb[common_to_3]*5/7).dropna()

common_to_2 = ['PaAtt', 'FL']
df_temp = (fd_qb[common_to_2]/6 + fp_qb[common_to_2]*5/6).dropna()

df3_qb['PaAtt'], df3_qb['FL'] = df_temp['PaAtt'], df_temp['FL']

df3_qb['DKPts'] = (fd_qb['DKPts']/7 + fp_qb['DKPts']*5/7 + swish_qb['DKPts']/7)
df3_qb['FDPts'] = (fd_qb['FDPts']/2 + swish_qb['FDPts']/2)

grand_means_qb = df3_qb.mean()
grand_stds_qb = df3_qb.std()

# Gotta do something about when just one site is missing a player. Right now they all need to have projections for it to work

In [9]:
#Standardize Everything Statistically

swish_qb[['DKPts','FDPts']] = StandardScaler().fit_transform(swish_qb[['DKPts','FDPts']])

fd_qb[['PaAtt', 'Cmp', 'PaYds', 'Int', 'PaTD', 'RuAtt', 'RuYds', 'RuTD', 'FL', 'DKPts', 'FDPts']] = \
StandardScaler().fit_transform(fd_qb[['PaAtt', 'Cmp', 'PaYds', 'Int', 'PaTD', 'RuAtt', 'RuYds', 'RuTD', 'FL', 'DKPts', 'FDPts']])

fp_qb[['PaAtt', 'Cmp', 'PaYds', 'Int', 'PaTD', 'RuAtt', 'RuYds', 'RuTD', 'FL', 'DKPts']] = \
StandardScaler().fit_transform(fp_qb[['PaAtt', 'Cmp', 'PaYds', 'Int', 'PaTD', 'RuAtt', 'RuYds', 'RuTD', 'FL', 'DKPts']])

fs_qb[['Cmp', 'PaYds', 'Int', 'PaTD', 'RuAtt', 'RuYds', 'RuTD']] = \
StandardScaler().fit_transform(fs_qb[['Cmp', 'PaYds', 'Int', 'PaTD', 'RuAtt', 'RuYds', 'RuTD']])

df3_qb = (fs_qb[common_to_3]/7 + fd_qb[common_to_3]/7 + fp_qb[common_to_3]*5/7).dropna()

common_to_2 = ['PaAtt', 'FL']
df_temp = (fd_qb[common_to_2]/6 + fp_qb[common_to_2]*5/6).dropna()

df3_qb['PaAtt'], df3_qb['FL'] = df_temp['PaAtt'], df_temp['FL']

df3_qb['DKPts'] = (fd_qb['DKPts']/7 + fp_qb['DKPts']*5/7 + swish_qb['DKPts']/7)
df3_qb['FDPts'] = (fd_qb['FDPts']/2 + swish_qb['FDPts']/2)

final_qb = df3_qb * grand_stds_qb + grand_means_qb

final_qb.head()

,Cmp,PaYds,Int,PaTD,RuYds,RuTD,RuAtt,PaAtt,FL,DKPts,FDPts
"(Aaron Rodgers, GB, QB)",24.895001,281.172312,0.758529,2.229557,19.434032,0.132421,4.043398,39.010379,0.201335,21.682212,22.722762
"(Alex Smith, KC, QB)",22.583566,254.940066,0.714144,1.327977,11.446307,0.086322,2.986888,35.632316,0.219654,15.431702,15.816232
"(Andy Dalton, CIN, QB)",23.108233,267.277428,0.819059,1.589498,12.713245,0.117357,3.508792,37.509878,0.183017,17.139278,17.774041
"(Ben Roethlisberger, PIT, QB)",25.377076,290.603461,0.999079,1.870150,5.491658,0.042527,1.591586,38.914390,0.278863,18.148119,19.783542
"(Blake Bortles, JAX, QB)",21.495543,240.173625,1.236910,1.563338,22.025793,0.114504,4.058500,37.590798,0.219654,16.605544,18.714019


In [10]:
############################################################
# RBs
############################################################

In [11]:
cols = 'name as PlayerName, primary_pos_abbr as Position, nickname as TeamName, home || " " || opp_abbr as Opponent,\
            dk_fpts as DKPts, dk_salary as DKSalary, fd_fpts as FDPts, fd_salary as FDSalary'

table_name = 'swish_week_{}'.format(week)

swish_rb = pd.read_sql_query("SELECT {s}\
    FROM {t} \
    WHERE primary_pos_abbr == 'RB' AND dk_fpts > 5 \
    ORDER BY PlayerName DESC".format(s=cols, t=table_name), engine)
        
swish_rb = standardize_names(swish_rb)
swish_rb.head()

,Opponent,DKPts,DKSalary,FDPts,FDSalary
"(Todd Gurley, LA, RB)",@ NYJ,13.2367,5300.0,11.7293,7000.0
"(Tim Hightower, NO, RB)",vs DEN,9.3211,4600.0,8.5797,6200.0
"(Terrance West, BAL, RB)",vs CLE,12.0601,5700.0,10.8408,6500.0
"(TJ Yeldon, JAX, RB)",vs HOU,10.2185,3700.0,8.7029,5500.0
"(Spencer Ware, KC, RB)",@ CAR,10.2492,6700.0,9.2589,7300.0


In [12]:
cols = 'Name as PlayerName, Position, Team as TeamName, \
            RushingAttempts as RuAtt, RushingYards as RuYds, RushingTouchdowns as RuTD, \
            Receptions as Rec, ReceivingYards as ReYds, ReceivingTouchdowns as ReTD, \
            FumblesLost as FL, FantasyPointsDraftKings as DKPts, FantasyPointsFanDuel as FDPts'

table_name = 'fantasy_data_off_week_{}'.format(week)

fd_rb = pd.read_sql_query("SELECT {s}\
    FROM {t} \
    WHERE Position == 'RB' AND FantasyPointsDraftKings > 5 \
    ORDER BY PlayerName DESC".format(s=cols, t=table_name), engine)

fd_rb = standardize_names(fd_rb)

fd_rb.head()

,RuAtt,RuYds,RuTD,Rec,ReYds,ReTD,FL,DKPts,FDPts
"(Todd Gurley, LA, RB)",19.9,71.1,0.5,3.6,29.9,0.1,0.2,16.9,14.8
"(Tim Hightower, NO, RB)",11.9,49.7,0.5,1.8,11.8,0.1,0.1,11.2,10.2
"(Terrance West, BAL, RB)",17.8,75.5,0.6,1.7,13.5,0.0,0.1,13.9,12.9
"(TJ Yeldon, JAX, RB)",9.1,38.1,0.2,2.6,18.3,0.1,0.1,10.1,8.7
"(Spencer Ware, KC, RB)",16.3,68.5,0.7,3.0,31.7,0.1,0.2,17.6,15.8


In [13]:
# FPts are PPR points in the db. Roughly equal to DK scoring
cols = 'Player as PlayerName, Team as TeamName, \
        ATT as RuAtt, YDS as RuYds, TDS as RuTD, \
        REC as Rec, YDSz as ReYds, TDSz as ReTD, \
        FL, FPTS as DKPts'

table_name = 'fantasy_pros_rb_week_{}'.format(week)

fp_rb = pd.read_sql_query("SELECT {s}\
    FROM {t} \
    WHERE FPTS > 5 \
    ORDER BY PlayerName DESC".format(s=cols, t=table_name), engine)

fp_rb['Position'] = 'RB'
fp_rb['FDPts'] = fp_rb['DKPts'] - 0.5 * fp_rb['Rec']
fp_rb = standardize_names(fp_rb)
fp_rb.head()

,RuAtt,RuYds,RuTD,Rec,ReYds,ReTD,FL,DKPts,FDPts
"(Ty Montgomery, GB, RB)",8.1,33.3,0.1,4.3,36.5,0.3,0.0,13.7,11.55
"(Todd Gurley, LA, RB)",18.0,62.1,0.3,3.0,23.2,0.1,0.2,13.4,11.90
"(Tim Hightower, NO, RB)",10.2,41.4,0.3,1.5,11.6,0.1,0.1,8.5,7.75
"(Terrance West, BAL, RB)",14.3,61.5,0.4,1.5,12.0,0.0,0.1,11.4,10.65
"(TJ Yeldon, JAX, RB)",7.5,29.0,0.1,2.6,19.5,0.1,0.1,8.5,7.20


In [14]:
cols = 'Name as PlayerName, Team as TeamName, Pos as Position, \
        Att as RuAtt, RushYards as RuYds, RushTD as RuTD, \
        Rec, RecYards as ReYds, RecTd as ReTD'

table_name = 'fantasy_sharks_week_{}'.format(week)

fs_rb = pd.read_sql_query("SELECT {s}\
    FROM {t} \
    WHERE FantasyPoints > 4 AND Pos == 'RB' \
    ORDER BY PlayerName DESC".format(s=cols, t=table_name), engine)

fs_rb['PlayerName'] = fs_rb['PlayerName'].apply(flip_first_last)
fs_rb = standardize_names(fs_rb)

fs_rb.head()

,RuAtt,RuYds,RuTD,Rec,ReYds,ReTD
"(TJ Yeldon, JAX, RB)",8.85,33,0.09,2.10,20,0.14
"(Damien Williams, MIA, RB)",3.48,15,0.25,1.61,18,0.04
"(James White, NE, RB)",4.00,14,0.07,4.00,40,0.27
"(Terrance West, BAL, RB)",14.96,59,0.39,2.21,13,0.04
"(Charcandrick West, KC, RB)",7.18,27,0.16,1.72,12,0.03


In [15]:
# These columns are common to all three dfs: ['Cmp', 'PaYds', 'Int', 'PaTD', 'RuAtt', 'RuYds', 'RuTD']
common_to_3 = list(fs_rb.columns)

# add the columns together which they have in common, drop any NaNs which should be shiite players
# also weight each accordingly, fp_qb gets 5 times the weight cuz its an aggregate of 5 sources
df3_rb = (fs_rb[common_to_3]/7 + fd_rb[common_to_3]/7 + fp_rb[common_to_3]*5/7).dropna()

common_to_2 = ['FL']
df_temp = (fd_rb[common_to_2]/6 + fp_rb[common_to_2]*5/6).dropna()

df3_rb['FL'] = df_temp['FL']

df3_rb['DKPts'] = (fd_rb['DKPts']/7 + fp_rb['DKPts']*5/7 + swish_rb['DKPts']/7)
df3_rb['FDPts'] = (fd_rb['FDPts']/7 + fp_rb['FDPts']*5/7 + swish_rb['FDPts']/7)

grand_means_rb = df3_rb.mean()
grand_stds_rb = df3_rb.std()

In [16]:
#Standardize Everything Statistically

swish_rb[['DKPts','FDPts']] = StandardScaler().fit_transform(swish_rb[['DKPts','FDPts']])

fd_rb[common_to_3] = StandardScaler().fit_transform(fd_rb[common_to_3])

fp_rb[common_to_3] = StandardScaler().fit_transform(fp_rb[common_to_3])

fs_rb[common_to_3] = StandardScaler().fit_transform(fs_rb[common_to_3])

df3_rb[common_to_3] = (fs_rb[common_to_3]/7 + fd_rb[common_to_3]/7 + fp_rb[common_to_3]*5/7).dropna()

common_to_2 = ['FL', 'DKPts', 'FDPts']
fd_rb[common_to_2] = StandardScaler().fit_transform(fd_rb[common_to_2])
fp_rb[common_to_2] = StandardScaler().fit_transform(fp_rb[common_to_2])

df_temp = (fd_rb[common_to_2]/6 + fp_rb[common_to_2]*5/6).dropna()

df3_rb['FL'] = df_temp['FL']

df3_rb['DKPts'] = (df_temp['DKPts']*6/7 + swish_rb['DKPts']/7)
df3_rb['FDPts'] = (df_temp['FDPts']*6/7 + swish_rb['FDPts']/7)

final_rb = df3_rb * grand_stds_rb + grand_means_rb

In [17]:
final_rb.head()

,RuAtt,RuYds,RuTD,Rec,ReYds,ReTD,FL,DKPts,FDPts
"(TJ Yeldon, JAX, RB)",8.488313,32.976807,0.127247,2.579471,19.888638,0.100500,0.106078,9.479617,8.092007
"(James White, NE, RB)",5.051033,19.383561,0.111719,4.030641,33.663679,0.185487,0.090658,11.043931,8.987198
"(Terrance West, BAL, RB)",15.316530,64.944795,0.439318,1.730806,12.991496,0.019281,0.106078,12.353368,11.465531
"(Charcandrick West, KC, RB)",6.648837,25.058589,0.137305,2.021322,15.064242,0.086683,0.106078,7.171224,6.159613
"(Spencer Ware, KC, RB)",15.450986,64.027889,0.521880,2.625129,26.743486,0.094220,0.202021,14.896034,13.483878


In [18]:
##################################################
# WRs
##################################################

In [19]:

cols = 'name as PlayerName, primary_pos_abbr as Position, nickname as TeamName, home || " " || opp_abbr as Opponent,\
            dk_fpts as DKPts, dk_salary as DKSalary, fd_fpts as FDPts, fd_salary as FDSalary'

table_name = 'swish_week_{}'.format(week)

swish_wr = pd.read_sql_query("SELECT {s}\
    FROM {t} \
    WHERE primary_pos_abbr == 'WR' AND dk_fpts > 5 \
    ORDER BY PlayerName DESC".format(s=cols, t=table_name), engine)
        
swish_wr = standardize_names(swish_wr)


cols = 'Name as PlayerName, Position, Team as TeamName, \
            RushingAttempts as RuAtt, RushingYards as RuYds, RushingTouchdowns as RuTD, \
            Receptions as Rec, ReceivingYards as ReYds, ReceivingTouchdowns as ReTD, \
            FumblesLost as FL, FantasyPointsDraftKings as DKPts, FantasyPointsFanDuel as FDPts'

table_name = 'fantasy_data_off_week_{}'.format(week)

fd_wr = pd.read_sql_query("SELECT {s}\
    FROM {t} \
    WHERE Position == 'WR' AND FantasyPointsDraftKings > 5 \
    ORDER BY PlayerName DESC".format(s=cols, t=table_name), engine)

fd_wr = standardize_names(fd_wr)


# FPts are PPR points in the db. Roughly equal to DK scoring
cols = 'Player as PlayerName, Team as TeamName, \
        ATT as RuAtt, YDS as RuYds, TDS as RuTD, \
        REC as Rec, YDSz as ReYds, TDSz as ReTD, \
        FL, FPTS as DKPts'

table_name = 'fantasy_pros_wr_week_{}'.format(week)

fp_wr = pd.read_sql_query("SELECT {s}\
    FROM {t} \
    WHERE FPTS > 5 \
    ORDER BY PlayerName DESC".format(s=cols, t=table_name), engine)

fp_wr['Position'] = 'WR'
fp_wr['FDPts'] = fp_wr['DKPts'] - 0.5 * fp_wr['Rec']
fp_wr = standardize_names(fp_wr)


cols = 'Name as PlayerName, Team as TeamName, Pos as Position, \
        Att as RuAtt, RushYards as RuYds, RushTD as RuTD, \
        Rec, RecYards as ReYds, RecTd as ReTD'

table_name = 'fantasy_sharks_week_{}'.format(week)

fs_wr = pd.read_sql_query("SELECT {s}\
    FROM {t} \
    WHERE FantasyPoints > 4 AND Pos == 'WR' \
    ORDER BY PlayerName DESC".format(s=cols, t=table_name), engine)

fs_wr['PlayerName'] = fs_wr['PlayerName'].apply(flip_first_last)
fs_wr = standardize_names(fs_wr)

# These columns are common to all three dfs: ['Cmp', 'PaYds', 'Int', 'PaTD', 'RuAtt', 'RuYds', 'RuTD']
common_to_3 = list(fs_wr.columns)

# add the columns together which they have in common, drop any NaNs which should be shiite players
# also weight each accordingly, fp_qb gets 5 times the weight cuz its an aggregate of 5 sources
df3_wr = (fs_wr[common_to_3]/7 + fd_wr[common_to_3]/7 + fp_wr[common_to_3]*5/7).dropna()

common_to_2 = ['FL']
df_temp = (fd_wr[common_to_2]/6 + fp_wr[common_to_2]*5/6).dropna()

df3_wr['FL'] = df_temp['FL']

df3_wr['DKPts'] = (fd_wr['DKPts']/7 + fp_wr['DKPts']*5/7 + swish_wr['DKPts']/7)
df3_wr['FDPts'] = (fd_wr['FDPts']/7 + fp_wr['FDPts']*5/7 + swish_wr['FDPts']/7)

grand_means_wr = df3_wr.mean()
grand_stds_wr = df3_wr.std()


#Standardize Everything Statistically

swish_wr[['DKPts','FDPts']] = StandardScaler().fit_transform(swish_wr[['DKPts','FDPts']])

fd_wr[common_to_3] = StandardScaler().fit_transform(fd_wr[common_to_3])

fp_wr[common_to_3] = StandardScaler().fit_transform(fp_wr[common_to_3])

fs_wr[common_to_3] = StandardScaler().fit_transform(fs_wr[common_to_3])

df3_wr[common_to_3] = (fs_wr[common_to_3]/7 + fd_wr[common_to_3]/7 + fp_wr[common_to_3]*5/7).dropna()

common_to_2 = ['FL', 'DKPts', 'FDPts']
fd_wr[common_to_2] = StandardScaler().fit_transform(fd_wr[common_to_2])
fp_wr[common_to_2] = StandardScaler().fit_transform(fp_wr[common_to_2])

df_temp = (fd_wr[common_to_2]/6 + fp_wr[common_to_2]*5/6).dropna()

df3_wr['FL'] = df_temp['FL']

df3_wr['DKPts'] = (df_temp['DKPts']*6/7 + swish_wr['DKPts']/7)
df3_wr['FDPts'] = (df_temp['FDPts']*6/7 + swish_wr['FDPts']/7)

final_wr = df3_wr * grand_stds_wr + grand_means_wr
final_wr.head()

,RuAtt,RuYds,RuTD,Rec,ReYds,ReTD,FL,DKPts,FDPts
"(AJ Green, CIN, WR)",-0.007270,-0.040882,0.000323,6.806463,94.923930,0.605581,0.083417,19.963981,16.416989
"(Adam Humphries, TB, WR)",0.097454,0.797194,0.000323,3.564661,45.230186,0.221032,0.001628,9.567768,7.702478
"(Adam Thielen, MIN, WR)",0.009129,0.117586,0.000323,3.324560,45.214112,0.243412,0.001628,9.444474,7.741027
"(Albert Wilson, KC, WR)",0.067013,0.521860,0.000323,3.161383,39.458614,0.205889,0.001628,8.008807,6.450409
"(Allen Hurns, JAX, WR)",-0.007270,-0.040882,0.000323,3.523518,48.191276,0.247366,0.001628,10.026097,8.243594


In [20]:
##################################################
# TE
##################################################

In [21]:

cols = 'name as PlayerName, primary_pos_abbr as Position, \
        nickname as TeamName, home || " " || opp_abbr as Opponent,\
        dk_fpts as DKPts, dk_salary as DKSalary, fd_fpts as FDPts, fd_salary as FDSalary'

table_name = 'swish_week_{}'.format(week)

swish_te = pd.read_sql_query("SELECT {s}\
    FROM {t} \
    WHERE primary_pos_abbr == 'TE' AND dk_fpts > 3 \
    ORDER BY PlayerName DESC".format(s=cols, t=table_name), engine)
        
swish_te = standardize_names(swish_te)


cols = 'Name as PlayerName, Position, Team as TeamName, \
            Receptions as Rec, ReceivingYards as ReYds, ReceivingTouchdowns as ReTD, \
            FumblesLost as FL, FantasyPointsDraftKings as DKPts, FantasyPointsFanDuel as FDPts'

table_name = 'fantasy_data_off_week_{}'.format(week)

fd_te = pd.read_sql_query("SELECT {s}\
    FROM {t} \
    WHERE Position == 'TE' AND FantasyPointsDraftKings > 3 \
    ORDER BY PlayerName DESC".format(s=cols, t=table_name), engine)

fd_te = standardize_names(fd_te)


# FPts are PPR points in the db. Roughly equal to DK scoring
cols = 'Player as PlayerName, Team as TeamName, \
        REC as Rec, YDS as ReYds, TDS as ReTD, \
        FL, FPTS as DKPts'

table_name = 'fantasy_pros_te_week_{}'.format(week)

fp_te = pd.read_sql_query("SELECT {s}\
    FROM {t} \
    WHERE FPTS > 3 \
    ORDER BY PlayerName DESC".format(s=cols, t=table_name), engine)

fp_te['Position'] = 'TE'
fp_te['FDPts'] = fp_te['DKPts'] - 0.5 * fp_te['Rec']
fp_te = standardize_names(fp_te)


cols = 'Name as PlayerName, Team as TeamName, Pos as Position, \
        Rec, RecYards as ReYds, RecTd as ReTD'

table_name = 'fantasy_sharks_week_{}'.format(week)

fs_te = pd.read_sql_query("SELECT {s}\
    FROM {t} \
    WHERE FantasyPoints > 3 AND Pos == 'TE' \
    ORDER BY PlayerName DESC".format(s=cols, t=table_name), engine)

fs_te['PlayerName'] = fs_te['PlayerName'].apply(flip_first_last)
fs_te = standardize_names(fs_te)

# These columns are common to all three dfs: ['Cmp', 'PaYds', 'Int', 'PaTD', 'RuAtt', 'RuYds', 'RuTD']
common_to_3 = list(fs_te.columns)

# add the columns together which they have in common, drop any NaNs which should be shiite players
# also weight each accordingly, fp_qb gets 5 times the weight cuz its an aggregate of 5 sources
df3_te = (fs_te[common_to_3]/7 + fd_te[common_to_3]/7 + fp_te[common_to_3]*5/7).dropna()

common_to_2 = ['FL']
df_temp = (fd_te[common_to_2]/6 + fp_te[common_to_2]*5/6).dropna()

df3_te['FL'] = df_temp['FL']

df3_te['DKPts'] = (fd_te['DKPts']/7 + fp_te['DKPts']*5/7 + swish_te['DKPts']/7)
df3_te['FDPts'] = (fd_te['FDPts']/7 + fp_te['FDPts']*5/7 + swish_te['FDPts']/7)

grand_means_te = df3_te.mean()
grand_stds_te = df3_te.std()


#Standardize Everything Statistically

swish_te[['DKPts','FDPts']] = StandardScaler().fit_transform(swish_te[['DKPts','FDPts']])

fd_te[common_to_3] = StandardScaler().fit_transform(fd_te[common_to_3])

fp_te[common_to_3] = StandardScaler().fit_transform(fp_te[common_to_3])

fs_te[common_to_3] = StandardScaler().fit_transform(fs_te[common_to_3])

df3_te[common_to_3] = (fs_te[common_to_3]/7 + fd_te[common_to_3]/7 + fp_te[common_to_3]*5/7).dropna()

common_to_2 = ['FL', 'DKPts', 'FDPts']
fd_te[common_to_2] = StandardScaler().fit_transform(fd_te[common_to_2])
fp_te[common_to_2] = StandardScaler().fit_transform(fp_te[common_to_2])

df_temp = (fd_te[common_to_2]/6 + fp_te[common_to_2]*5/6).dropna()

df3_te['FL'] = df_temp['FL']

df3_te['DKPts'] = (df_temp['DKPts']*6/7 + swish_te['DKPts']/7)
df3_te['FDPts'] = (df_temp['FDPts']*6/7 + swish_te['FDPts']/7)

final_te = df3_te * grand_stds_te + grand_means_te
final_te.head()

,Rec,ReYds,ReTD,FL,DKPts,FDPts
"(Jason Witten, DAL, TE)",4.850988,51.242829,0.239920,0.000772,11.797031,9.338002
"(Delanie Walker, TEN, TE)",4.987041,62.342428,0.509018,0.000772,14.274169,11.736532
"(Will Tye, NYG, TE)",3.482491,35.236814,0.136715,0.000772,8.503966,6.690553
"(Julius Thomas, JAX, TE)",3.443722,37.744189,0.269866,0.000772,9.124019,7.311168
"(Dion Sims, MIA, TE)",2.090662,22.472876,0.144218,0.000772,NaN,NaN


In [22]:

for df, swish_df, pos in [(final_qb,swish_qb,'qb'), (final_rb,swish_rb,'rb'), \
                (final_wr,swish_wr,'wr'), (final_te,swish_te,'te')]:
    
    df = pd.merge(df,swish_df[['Opponent','DKSalary','FDSalary']], \
                  left_index=True,right_index=True)
    col_list = ['Name','Team','Position'] + list(df.columns)
    df['Name'] = [name for name,team,position in tuple(df.index)]
    df['Team'] = [team for name,team,position in tuple(df.index)]
    df['Position'] = [position for name,team,position in tuple(df.index)]
    df = df[col_list]
    df['DKValue'] = df['DKPts'] * 1000 / df['DKSalary']
    
    if pos == 'qb':
        df['ImpliedPts'] = (df['DKSalary']/1000)*2.337 + 2.2198
    if pos == 'rb':
        df['ImpliedPts'] = (df['DKSalary']/1000)*2.8394 - 3.746
    if pos == 'wr':
        df['ImpliedPts'] = (df['DKSalary']/1000)*2.2928 - 0.9003
    if pos == 'te':
        df['ImpliedPts'] = (df['DKSalary']/1000)*1.9 - 0.37
        
    df['PlusMinus'] = df['DKPts'] - df['ImpliedPts']
    sql_table_name = 'aggregate_{}_week_{}'.format(pos, week)
    
    df.to_sql(sql_table_name, engine, index=False, if_exists='replace')